In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# Set random seed for reproducibility (optional)
random.seed(42)
np.random.seed(42)

# Fixed handoff height for attached rectangles
handoff_height = 0.1

def create_title_with_worker_assignments(W):
    """
    Create a title string showing task sequence grouped by workers.
    Example: Job Design [(1)(2)][(3)(4)][(5)(6)] for 6 tasks split between 3 workers.
    """
    current_worker = W[0]
    groups = []
    current_group = []

    for task_idx, worker in enumerate(W, 1):
        if worker != current_worker:
            groups.append(current_group)
            current_group = []
            current_worker = worker
        current_group.append(str(task_idx))
    groups.append(current_group)

    worker_sections = [
        "[" + "".join(f"({task})" for task in group) + "]" for group in groups
    ]
    return "Job Design " + "".join(worker_sections)

def draw_rect_square_unit(ax, x, y, t, c, h, task_idx):
    """
    Draw a single rectangle unit with an attached rectangle at position (x,y).
    The main rectangle has width t and height c.
    The attached rectangle has fixed height (handoff_height) and width h.
    """
    # Draw main rectangle (task)
    rect = plt.Rectangle((x, y), t, c, fill=False, color="blue", linewidth=1)
    ax.add_patch(rect)

    # Add task index at the center
    center_x = x + t / 2
    center_y = y + c / 2
    ax.text(center_x, center_y, str(task_idx),
            horizontalalignment="center", verticalalignment="center")

    # Draw attached rectangle
    attached_x = x + t
    attached_top_y = y + c
    attached_rect = plt.Rectangle(
        (attached_x, attached_top_y - handoff_height),  # lower left corner adjusted for handoff_height
        h,               # width given by h
        handoff_height,  # fixed height
        fill=False,
        color="#FF9999",
        linestyle="--",
        linewidth=1,
    )
    ax.add_patch(attached_rect)

    # The next starting point is at the top right of the attached rectangle.
    next_pos = (attached_x + h, attached_top_y)
    coords = [
        (attached_x, attached_top_y),
        (attached_x + h, attached_top_y - handoff_height),
    ]
    return next_pos, coords

def draw_rect_square_sequence(T, C, H, W):
    """
    Draw a sequence of rectangle units with:
      - A main rectangle (dimensions from T and C)
      - An attached rectangle (width from H and fixed height)
    Tasks are grouped by worker assignments (W).
    """
    if not (len(T) == len(C) == len(H) == len(W)):
        raise ValueError("All input vectors must have the same length")

    fig, ax = plt.subplots(figsize=(10, 8))
    current_pos = (0, 0)
    all_coords = [(0, 0)]
    x_positions = [0]
    worker_sections = []
    current_section_start_x = 0
    current_section_start_y = 0
    current_worker = W[0]
    current_worker_T = []
    current_worker_C = []
    transition_rects = []

    for i, (t, c, h, w) in enumerate(zip(T, C, H, W)):
        next_pos, new_coords = draw_rect_square_unit(ax, current_pos[0], current_pos[1], t, c, h, i + 1)
        if i < len(T) - 1 and w == W[i + 1]:
            next_pos = (next_pos[0] - h, next_pos[1])
        current_worker_T.append(t)
        current_worker_C.append(c)
        if (i < len(T) - 1 and w != W[i + 1]) or i == len(T) - 1:
            if i < len(T) - 1 and w != W[i + 1]:
                transition_rects.append((current_pos[0] + t, current_pos[1] + c, h, sum(current_worker_C), i + 1))
            worker_sections.append(
                (
                    current_section_start_x,
                    current_section_start_y,
                    next_pos[0],
                    current_worker,
                    sum(current_worker_T),
                    sum(current_worker_C),
                )
            )
            x_positions.append(next_pos[0])
            if i < len(T) - 1:
                current_section_start_x = next_pos[0]
                current_section_start_y = next_pos[1]
                current_worker = W[i + 1]
                current_worker_T = []
                current_worker_C = []
        current_pos = next_pos
        all_coords.extend(new_coords)

    all_coords = np.array(all_coords)
    x_min, y_min = all_coords.min(axis=0)
    x_max, y_max = all_coords.max(axis=0)
    max_y = y_max + (y_max - y_min) * 0.1
    for start_x, start_y, end_x, worker_id, total_t, total_c in worker_sections:
        worker_box = plt.Rectangle(
            (start_x, start_y),
            total_t,
            total_c,
            fill=True,
            facecolor="blue",
            edgecolor="blue",
            linestyle="--",
            linewidth=1,
            alpha=0.2,
        )
        ax.add_patch(worker_box)
    for x, y, h, total_c, task_idx in transition_rects:
        transition_rect = plt.Rectangle(
            (x, y - total_c),
            h,
            total_c,
            fill=True,
            facecolor="pink",
            edgecolor="#FF9999",
            linestyle="--",
            linewidth=1,
            alpha=0.4,
        )
        ax.add_patch(transition_rect)
        center_x = x + h / 2
        center_y = y - total_c / 2
        ax.text(center_x, center_y, f"$h_{{{task_idx}}}$",
                horizontalalignment="center", verticalalignment="center", fontsize=10)

    padding = max(x_max - x_min, y_max - y_min) * 0.1
    ax.set_xlim(x_min - padding, x_max + padding)
    ax.set_ylim(y_min - padding, max_y + padding)
    ax.set_aspect("equal")
    ax.set_title(create_title_with_worker_assignments(W))
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])

    return fig, ax

def generate_parameters(num_jobs):
    """
    Generate a sensible set of parameters (T, C, H, W) for the given number of jobs.
    
    - T: Random main rectangle lengths (0.5 to 3.0)
    - C: Random main rectangle heights (1.0 to 3.0)
    - H: Random attached rectangle widths (0 to 2.5, with the last value set to 0)
    - W: Worker assignments as a weakly increasing sequence starting at 1,
         with each value ≤ num_jobs.
    """
    T = np.round(np.random.uniform(0.25, 5, size=num_jobs), 1)
    C = np.round(np.random.uniform(0.25, 5, size=num_jobs), 1)
    H = np.round(np.random.uniform(0, 5, size=num_jobs), 1)
    H[-1] = 0  # Ensure the last attached rectangle width is 0 (as in your original example)
    
    # Build a weakly increasing worker assignment array with increased variation.
    W = [1]
    increment_probability = 0.6  # Adjust this value to control variation
    for i in range(1, num_jobs):
        if W[-1] < num_jobs:
            if np.random.rand() < increment_probability:
                W.append(W[-1] + 1)
            else:
                W.append(W[-1])
        else:
            W.append(W[-1])
    return T, C, H, np.array(W)

# Loop over job counts and generate 5 examples for each.
job_counts = [5, 6, 7, 8, 9, 10, 11, 12, 13]
for num_jobs in job_counts:
    for example in range(1, 11):
        T, C, H, W = generate_parameters(num_jobs)
        fig, ax = draw_rect_square_sequence(T, C, H, W)
        # Update the title to include job count and example number
        ax.set_title(create_title_with_worker_assignments(W) + f"    (Tasks: {num_jobs}, Example: {example})")
        # Save the figure with a unique filename
        plt.savefig(f"../writeup/plots/job_design/job_design_{num_jobs}tasks_{example}.png", dpi=300)
        plt.close(fig)

<Figure size 640x480 with 0 Axes>